In [ ]:
#This should be run in Jupyter
#Before it will run, sqlalchemy and mysql-connecter should be installed using Anaconda command prompt:
#conda install sqlalchemy
#conda install -c anaconda mysql-connector-python

import sqlalchemy as sqla
from sqlalchemy import create_engine
import traceback
import glob
import os
from pprint import pprint
import json
import requests
import time
from IPython.display import display
import mysql.connector

#set up the database connection
URI = "database-1.cx36tayg9smy.us-east-1.rds.amazonaws.com"
PORT = "3306"
DB = "SE_Project"
USER = "admin"
PASSWORD = "liamstacy"

engine = create_engine("mysql+mysqlconnector://{}:{}@{}:{}/{}".format(USER, PASSWORD, URI, PORT, DB), echo=True)

#Retreive the json data
APIKEY = "eb2ccb1179e38c5251aac92c5182497cd4e7222d"
NAME = "Dublin"
STATIONS_URI = "https://api.jcdecaux.com/vls/v1/stations"

r = requests.get(STATIONS_URI, params={"apiKey": APIKEY, "contract": NAME})

#Create the Stations table
sql = """CREATE TABLE IF NOT EXISTS station (
        address VARCHAR(256),
        banking INTEGER,
        bike_stands INTEGER,
        bonus INTEGER,
        contract_name VARCHAR(256),
        name VARCHAR(256),
        number INTEGER,
        position_lat REAL,
        position_lng REAL,
        status VARCHAR(256))"""

try:
    res = engine.execute("DROP TABLE IF EXISTS station")
    res = engine.execute(sql)
    #print(res.fetchall())
except Exception as e:
    print(e)

#create a function to insert the data to the table
def stations_to_db(text):
    stations = json.loads(text)
    for station in stations:
        vals = (station.get("address"), int(station.get("banking")), station.get("bike_stands"), int(station.get("bonus")),
               station.get("contract_name"), station.get("name"), station.get("number"), station.get("position").get("lat"),
               station.get("position").get("lng"), station.get("status"))
        
        engine.execute("insert into station values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", vals)
        #break
    return

stations_to_db(r.text)
    
